In [70]:
!pip3 install google-generativeai python-dotenv pypdf pillow


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [71]:
!pip3 freeze > requirements.txt

In [72]:
import os
from dotenv import load_dotenv
from pypdf import PdfReader

In [73]:
load_dotenv(override=True)

gemini_api_key = os.getenv("GOOGLE_API_KEY")

if not gemini_api_key:
    raise ValueError("GOOGLE_API_KEY is not set in the environment variables.")

In [74]:
def read_pdf(file_path):
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error reading PDF file: {e}")
        return None

In [75]:
dietary_guidelines_text = read_pdf("resources/dietary_guidelines.pdf")

if not dietary_guidelines_text:
    raise ValueError("Failed to read the dietary guidelines PDF file.")

In [76]:
dietary_guidelines_text[:10]

'Dietary\nGu'

In [77]:
photo_analysis_system_prompt = """
You are a nutrition-label analyzer. From the provided product photo(s), extract ONLY the following fields in JSON:
- fat, cholesterol, sodium, carbohydrates, protein

Rules:
- Each field is a dictionary of key→value pairs you can read from the label 
  (e.g., {"total": 7, "saturated": 1.5, "unit": "g", "basis": "per_serving", "percent_dv": 9}).
- If a field is not readable or missing, set numeric values to 0 and strings to empty string "".
- Keep numbers as plain numbers (no % or unit symbols in the numbers themselves).
- If the label shows only percent daily value, put it as a number in "percent_dv".
- Do not convert salt↔sodium; only report what's printed.
- Output ONLY a JSON object matching the schema. No extra text.

Example for missing/unreadable values:
{"total": 0, "saturated": 0, "unit": "", "basis": "", "percent_dv": 0}
"""

In [78]:
import google.generativeai as genai
from google.generativeai.types import GenerationConfig
from PIL import Image

In [79]:
genai.configure(api_key=gemini_api_key)

In [80]:
nutrition_schema = {
    "type": "object",
    "properties": {
        "schema_version": {
            "type": "string"
        },
        "fat": {
            "type": "object",
            "properties": {
                "amount": {"type": "number"},
                "unit": {"type": "string"},
                "daily_value_percent": {"type": "number"}
            }
        },
        "cholesterol": {
            "type": "object",
            "properties": {
                "amount": {"type": "number"},
                "unit": {"type": "string"},
                "daily_value_percent": {"type": "number"}
            }
        },
        "sodium": {
            "type": "object",
            "properties": {
                "amount": {"type": "number"},
                "unit": {"type": "string"},
                "daily_value_percent": {"type": "number"}
            }
        },
        "carbohydrates": {
            "type": "object",
            "properties": {
                "amount": {"type": "number"},
                "unit": {"type": "string"},
                "daily_value_percent": {"type": "number"}
            }
        },
        "protein": {
            "type": "object",
            "properties": {
                "amount": {"type": "number"},
                "unit": {"type": "string"},
                "daily_value_percent": {"type": "number"}
            }
        }
    },
    "required": ["schema_version"]
}

In [81]:
model = genai.GenerativeModel('gemini-2.0-flash-exp')

In [82]:
generation_config = GenerationConfig(
    response_mime_type="application/json",
    response_schema=nutrition_schema
)

In [83]:
image = Image.open("resources/composition.jpg")

In [84]:
response = model.generate_content(
    [photo_analysis_system_prompt, "Analyze this nutrition label.", image],
    generation_config=generation_config
)

In [85]:
response_json = response.text
response_json

'{\n  "schema_version": "0",\n  "fat": {\n    "amount": 1.5,\n    "daily_value_percent": 2,\n    "unit": "g"\n  },\n  "protein": {\n    "amount": 4,\n    "daily_value_percent": 8,\n    "unit": "g"\n  },\n  "sodium": {\n    "amount": 200,\n    "daily_value_percent": 9,\n    "unit": "mg"\n  }\n}'